In [12]:
import pandas as pd

df = pd.read_csv("https://ec.europa.eu/eurostat/databrowser-backend/api/extraction/1.0/LIVE/true/sdmx/csv/HBS_STR_T223?i")


In [13]:
df['geo'].unique()

array(['AT', 'BE', 'BG', 'CY', 'CZ', 'DE', 'DK', 'EA', 'EA12', 'EA13',
       'EA17', 'EA18', 'EE', 'EEA28', 'EEA30_2007', 'EFTA', 'EL', 'ES',
       'EU15', 'EU25', 'EU27_2007', 'EU27_2020', 'FI', 'FR', 'HR', 'HU',
       'IE', 'IT', 'LT', 'LU', 'LV', 'ME', 'MK', 'MT', 'NL', 'NO', 'PL',
       'PT', 'RO', 'RS', 'SE', 'SI', 'SK', 'TR', 'UK', 'XK'], dtype=object)

In [14]:
df['coicop'].unique()

array(['CP01', 'CP011', 'CP0111', 'CP0112', 'CP0113', 'CP0114', 'CP0115',
       'CP0116', 'CP0117', 'CP0118', 'CP0119', 'CP012', 'CP0121',
       'CP0122', 'CP02', 'CP021', 'CP0211', 'CP0212', 'CP0213', 'CP022',
       'CP023', 'CP03', 'CP031', 'CP0311', 'CP0312', 'CP0313', 'CP0314',
       'CP032', 'CP0321', 'CP0322', 'CP04', 'CP041', 'CP0411', 'CP0412',
       'CP042', 'CP0421', 'CP0422', 'CP043', 'CP0431', 'CP0432', 'CP044',
       'CP0441', 'CP0442', 'CP0443', 'CP0444', 'CP045', 'CP0451',
       'CP0452', 'CP0453', 'CP0454', 'CP0455', 'CP05', 'CP051', 'CP0511',
       'CP0512', 'CP0513', 'CP052', 'CP053', 'CP0531', 'CP0532', 'CP0533',
       'CP054', 'CP055', 'CP0551', 'CP0552', 'CP056', 'CP0561', 'CP0562',
       'CP06', 'CP061', 'CP0611', 'CP0612', 'CP0613', 'CP062', 'CP0621',
       'CP0622', 'CP0623', 'CP063', 'CP07', 'CP071', 'CP0711', 'CP0712',
       'CP0713', 'CP072', 'CP0721', 'CP0722', 'CP0723', 'CP0724', 'CP073',
       'CP0731', 'CP0732', 'CP0733', 'CP0734', 'CP0735', 

In [16]:
# Keep only the rows where TIME_PERIOD is equal to 2015
df = df[df['TIME_PERIOD'] == 2015]
